In [1]:
import numpy as np
import pandas as pd
import os
import random
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, models
from PIL import Image
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
from information_processing import load_partitions, load_attributes
from celebA_class import CelebAAttributionDataset
from model_training import train_model_attribution, plot_training_history

In [2]:
torch.manual_seed(42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
PARENT_DIRNAME = os.path.expanduser("~/image-processing-project/")
IMAGE_DIR = os.path.join(PARENT_DIRNAME, "data/img_align_celeba/")
ATTR_DIR = os.path.join(PARENT_DIRNAME, "data/list_attr_celeba.csv")
EVAL_PARTITION_DIR = os.path.join(PARENT_DIRNAME, "data/list_eval_partition.csv")
MODEL_DIR = os.path.join(PARENT_DIRNAME, "fine_tuning/model")

In [4]:
# Hyperparameters
IMG_SIZE = 224
BATCH_SIZE = 32
NUM_EPOCHS = 10
LEARNING_RATE = 0.001

In [5]:
attributes, attr_names = load_attributes(ATTR_DIR)

Loaded 202599 images with 40 attributes.


In [6]:
partitions = load_partitions(EVAL_PARTITION_DIR)

Loaded 202599 images with partitions.


In [7]:
train_transforms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    transforms.RandomErasing(p=0.75, scale=(0.01, 0.3), ratio=(1.0, 1.0), value=0, inplace =True)
])

test_transforms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [ ]:
# Because we are not using the validation set for hypertunning, we can concatenate the training and validation sets
train_dataset = CelebAAttributionDataset(IMAGE_DIR, attributes, partitions, partition_type=0, transform=train_transforms)
val_dataset = CelebAAttributionDataset(IMAGE_DIR, attributes, partitions, partition_type=1, transform=train_transforms)
test_dataset = CelebAAttributionDataset(IMAGE_DIR, attributes, partitions, partition_type=2, transform=test_transforms)
# Concatenate the training and validation sets
final_train_dataset = torch.utils.data.ConcatDataset([train_dataset, val_dataset])

train_loader = DataLoader(final_train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

print(f"Training samples: {len(final_train_dataset)}\nTesting samples: {len(test_dataset)}")

In [9]:
# Load MobileNet pre-trained model
model = models.mobilenet_v2(pretrained=True)

/home/letruongzzio/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/letruongzzio/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [10]:
# Freeze all layers except the last 10
for param in model.features[:-10].parameters():
    param.requires_grad = False

# Fine-tune classifier: 
num_features = model.last_channel
model.classifier = nn.Sequential(
    nn.Linear(num_features, 512),
    nn.ReLU(),
    nn.Dropout(0.4),
    nn.Linear(512, len(attr_names)),
    nn.Sigmoid()
)

model = model.to(device)

In [11]:
# Loss function and Optimizer
criterion = nn.BCELoss()  # Binary Cross-Entropy Loss for multi-label classification
# Only parameters that require gradients are optimized
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=LEARNING_RATE)

In [ ]:
trained_model, history = train_model_attribution(model, train_loader, test_loader, criterion, optimizer, NUM_EPOCHS, device)
plot_training_history(history)

In [ ]:
torch.save(trained_model.state_dict(), MODEL_DIR + "/celeba_mobilenet_v2.pth")
print(f"Model saved to {MODEL_DIR}/celeba_mobilenet_v2.pth")